### **Mask R-CNN : image inference practice**

In [ ]:
!mkdir ./image
!wget -O ./image/beatles01.jpg https://github.com/chulminkw/DLCV/blob/master/data/image/beatles01.jpg?raw=true

In [ ]:
import cv2

image = cv2.imread('./image/beatles01.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
!mkdir ./pretrained
!wget -O ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
!wget -O ./pretrained/config_mask_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt

--2022-08-12 16:41:04--  http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.194.128, 2607:f8b0:4001:c5a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.194.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177817887 (170M) [application/x-tar]
Saving to: ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’

./pretrained/mask_r 100%[===================>] 169.58M   167MB/s    in 1.0s    

2022-08-12 16:41:05 (167 MB/s) - ‘./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [177817887/177817887]

--2022-08-12 16:41:05--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/mask_rcnn_inception_v2_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.co

In [ ]:
!tar -xvf ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz -C ./pretrained

mask_rcnn_inception_v2_coco_2018_01_28/
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
mask_rcnn_inception_v2_coco_2018_01_28/checkpoint
mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [ ]:
!ls -lia ./pretrained/mask_rcnn_inception_v2_coco_2018_01_28

total 132684
5242889 drwxr-xr-x 3 345018 5000     4096 Feb  1  2018 .
5242886 drwxr-xr-x 3 root   root     4096 Aug 12 16:41 ..
5242891 -rw-r--r-- 1 345018 5000       77 Feb  1  2018 checkpoint
5242900 -rw-r--r-- 1 345018 5000 67138064 Feb  1  2018 frozen_inference_graph.pb
5242893 -rw-r--r-- 1 345018 5000 63616508 Feb  1  2018 model.ckpt.data-00000-of-00001
5242890 -rw-r--r-- 1 345018 5000    16123 Feb  1  2018 model.ckpt.index
5242896 -rw-r--r-- 1 345018 5000  5069133 Feb  1  2018 model.ckpt.meta
5242892 -rw-r--r-- 1 345018 5000     3776 Feb  1  2018 pipeline.config
5242897 drwxr-xr-x 3 345018 5000     4096 Feb  1  2018 saved_model


In [ ]:
labels_to_names_seq = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                       10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                       20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                       30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                       40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                       50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                       60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                       70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                       80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                       90:'hair brush'}

In [ ]:
colors = list(
    [[0, 255, 0],
     [0, 0, 255],
     [255, 0, 0],
     [0, 255, 255],
     [255, 255, 0],
     [255, 0, 255],
     [80, 70, 180],
     [250, 80, 190],
     [245, 145, 50],
     [70, 150, 250],
     [50, 190, 190]])

In [ ]:
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb', 
                                       './pretrained/config_mask_graph.pbtxt')

cv_net.setInput(cv2.dnn.blobFromImage(image, swapRB=True, crop=False))

In [ ]:
boxes_out, masks_out = cv_net.forward(['detection_out_final', 'detection_masks'])
print(f'boxes_out shape : {boxes_out.shape} / masks_out shape : {masks_out.shape}')

boxes_detections_nums = boxes_out.shape[2]
masks_classes_nums = masks_out.shape[1]
print(f'boxes_detections_nums : {boxes_detections_nums} / masks_classes_nums : {masks_classes_nums}')

boxes_out shape : (1, 1, 100, 7) / masks_out shape : (100, 90, 15, 15)
boxes_detections_nums : 100 / masks_classes_nums : 90


In [ ]:
draw_image = image_rgb.copy()
draw_image_height = draw_image.shape[0]
draw_image_width = draw_image.shape[1]
print(f'draw_image shape : {draw_image.shape} / image_height : {draw_image_height} / image_width : {draw_image_width}')

draw_image shape : (633, 806, 3) / image_height : 633 / image_width : 806


In [ ]:
green_color=(0, 255, 0)
red_color=(0, 0, 255)

confidence_threshold = 0.5
mask_threshold = 0.3

#for i in range(boxes_detections_nums):
for i in range(1):
    box = boxes_out[0, 0, i, :]
    mask = masks_out[i, :, :, :]
    class_id = int(box[1])
    confidence_score = box[2]    
    if confidence_score > confidence_threshold:
        x1 = int(draw_image_width * box[3])
        y1 = int(draw_image_height * box[4])
        x2 = int(draw_image_width * box[5])
        y2 = int(draw_image_height * box[6])
        text = "{}: {:.4f}".format(labels_to_names_seq[class_id], confidence_score)
        cv2.rectangle(draw_image, (x1, y1), (x2, y2), green_color, thickness=2)
        cv2.putText(draw_image, text, (x1, y1-3), cv2.FONT_HERSHEY_SIMPLEX, 0.3, red_color, 1)

        class_mask_detected = mask[class_id]
        print('class_mask_detected shape:', class_mask_detected.shape)
        class_mask_scaled = cv2.resize(class_mask_detected, (x2-x1+1, y2-y1+1))
        print('class_mask_scaled shape:', class_mask_scaled.shape)
        class_mask_scaled_bool = (class_mask_scaled > mask_threshold)
        print('class_mask_scaled_bool shape:', class_mask_scaled_bool.shape)
        before_masking_roi = draw_image[y1:y2+1, x1:x2+1]
        print('before_masking_roi:', before_masking_roi.shape)
        make_mask = (class_mask_scaled_bool*255).astype('uint8')
        print('make_mask shape:', make_mask.shape)
        bitwise_masking_roi = cv2.bitwise_and(before_masking_roi, before_masking_roi, mask=make_mask)
        print('bitwise_masking_roi :', bitwise_masking_roi.shape)

class_mask_detected shape: (15, 15)
class_mask_scaled shape: (284, 140)
class_mask_scaled_bool shape: (284, 140)
before_masking_roi: (284, 140, 3)
make_mask shape: (284, 140)
bitwise_masking_roi : (284, 140, 3)


In [ ]:
import numpy as np

colorIndex = np.random.randint(0, len(colors)-1)
color = colors[colorIndex]

draw_image = image_rgb.copy()
after_masking_roi_color = draw_image[y1:y2+1, x1:x2+1]
after_masking_roi_bool = after_masking_roi_color[class_mask_scaled_bool]
after_masking_roi_color[class_mask_scaled_bool] = ([0.3*color[0], 0.3*color[1], 0.3*color[2]] + 0.6 * after_masking_roi_bool).astype('uint8')

after_masking_roi_contour = after_masking_roi_color.copy()
after_masking_roi_bn = class_mask_scaled_bool.astype('uint8')
contours, hierarchy = cv2.findContours(after_masking_roi_bn, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
after_masking_roi_contour = cv2.drawContours(after_masking_roi_contour, contours, -1, (255, 0, 0), 4)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(figsize=(22, 8), ncols=8, nrows=1)
axs[0].set_title('class_mask_detected')
axs[0].axis('off')
axs[0].imshow(class_mask_detected)

axs[1].set_title('class_mask_scaled')
axs[1].axis('off')
axs[1].imshow(class_mask_scaled)

axs[2].set_title('class_mask_scaled_bool')
axs[2].axis('off')
axs[2].imshow(class_mask_scaled_bool)

axs[3].set_title('before_masking_roi')
axs[3].axis('off')
axs[3].imshow(before_masking_roi)

axs[4].set_title('make_mask')
axs[4].axis('off')
axs[4].imshow(make_mask, cmap='gray')

axs[5].set_title('bitwise_masking_roi')
axs[5].axis('off')
axs[5].imshow(bitwise_masking_roi)

axs[6].set_title('after_masking_roi_color')
axs[6].axis('off')
axs[6].imshow(after_masking_roi_color)

axs[7].set_title('after_masking_roi_contour')
axs[7].axis('off')
axs[7].imshow(after_masking_roi_contour)
plt.show()